### Maven Everest Challenge

Use data storytelling to visualize the evolution of mankind's pursuit of the world's highest peak.

#### Challenge Objective:

For the Maven Everest Challenge, you’ll play the role of a data journalist tasked with telling the story of mankind’s quest to conquer Mount Everest. Using real expedition data, your goal is to craft a compelling visual narrative that highlights things like key milestones, shifting strategies, and the climbers who dared to reach the top of the world.

#### About The Data Set:

This dataset, based on the archives of Elizabeth Hawley, provides a comprehensive record of mountaineering expeditions in the Nepalese Himalaya, spanning from 1905 to 2024. It includes detailed information on 89,000+ members across 11,000+ expeditions and 480 mountain peaks, including dates, successes, and significant events.

In [143]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# show full output
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [144]:
# read all the datasets
expedition = pd.read_csv('E:\Maven_Analytics_Challenges\Everest_Challenge\HimalayanExpeditions\exped.csv')
members = pd.read_csv('E:\Maven_Analytics_Challenges\Everest_Challenge\HimalayanExpeditions\members.csv')
peaks = pd.read_csv('E:\Maven_Analytics_Challenges\Everest_Challenge\HimalayanExpeditions\peaks.csv')
reference = pd.read_csv('E:\\Maven_Analytics_Challenges\\Everest_Challenge\\HimalayanExpeditions\\refer.csv', encoding="ISO-8859-1")

In [145]:
print('Expedition shape:', expedition.shape)
print('Members shape:', members.shape)
print('Peaks shape:', peaks.shape)
print('Reference shape:', reference.shape)

Expedition shape: (11425, 65)
Members shape: (89000, 61)
Peaks shape: (480, 23)
Reference shape: (15586, 12)


In [146]:
# let's join the members and expedition dataframes on expid
members_expedition = pd.merge(members, expedition, on='expid', how='left')

In [147]:
members_expedition.shape

(89089, 125)

In [148]:
# lets check if values in peakid_x and peakid_y are same
for i in range(len(members_expedition)):
    if members_expedition['peakid_x'][i] != members_expedition['peakid_y'][i]:
        print('Not same:', members_expedition['peakid_x'][i], members_expedition['peakid_y'][i])

In [149]:
# drop the column peakid_y
members_expedition.drop(columns=['peakid_y'], inplace=True)

In [150]:
# now let's join the members_expedition dataframe with the peaks dataframe on peakid
members_expedition = pd.merge(members_expedition, peaks, left_on='peakid_x', right_on='peakid', how='left')

In [151]:
members_expedition.shape

(89089, 147)

In [152]:
# now finally let's join the members_expedition dataframe with the reference dataframe on expid
members_expedition = pd.merge(members_expedition, reference, on='expid', how='left')

In [153]:
members_expedition.shape

(200037, 158)

In [154]:
# let's remove the columns that has more than 90% null values
members_expedition = members_expedition.dropna(thresh=len(members_expedition) * 0.9, axis=1)

In [155]:
members_expedition.shape

(200037, 107)

In [156]:
# duplicate records
members_expedition.duplicated().sum()

110948

In [157]:
# lets remove the duplicate records
members_expedition = members_expedition.drop_duplicates().reset_index(drop=True)
members_expedition.shape

(89089, 107)

In [158]:
# colums with null values
members_expedition.columns[members_expedition.isnull().any()]

Index(['fname', 'lname', 'citizen', 'msmtterm', 'route1', 'leaders', 'sponsor',
       'smtdate', 'campsites', 'location', 'pyear', 'pseason', 'pmonth',
       'pday', 'pexpid', 'pcountry', 'psummiters'],
      dtype='object')

In [159]:
# let's fill the null values in the columns with "not available"
members_expedition = members_expedition.fillna('not available')

In [160]:
members_expedition.duplicated().sum()

0

In [161]:
# colums with null values
members_expedition.columns[members_expedition.isnull().any()]

Index([], dtype='object')

In [162]:
# lets check if values in peakid_x and peakid are same
for i in range(len(members_expedition)):
    if members_expedition['peakid_x'][i] != members_expedition['peakid'][i]:
        print('Not same:', members_expedition['peakid_x'][i], members_expedition['peakid'][i])

In [163]:
# let's drop the peakid_x column
members_expedition.drop(columns=['peakid_x'], inplace=True)

In [164]:
members_expedition.dtypes

expid         object
membid         int64
myear          int64
mseason       object
fname         object
lname         object
sex           object
citizen       object
status        object
leader          bool
deputy          bool
bconly          bool
nottobc         bool
support         bool
disabled        bool
hired           bool
sherpa          bool
tibetan         bool
msuccess        bool
mclaimed        bool
mdisputed       bool
msolo           bool
mtraverse       bool
mski            bool
mparapente      bool
mspeed          bool
mhighpt         bool
mroute1        int64
mroute2        int64
mroute3        int64
mascent1       int64
mascent2       int64
mascent3       int64
mo2used         bool
mo2none         bool
mo2climb        bool
mo2descent      bool
mo2sleep        bool
mo2medical      bool
death           bool
deathhgtm      int64
msmtbid       object
msmtterm      object
mchksum        int64
year           int64
season        object
host          object
route1       

In [171]:
members_expedition['citizen'].value_counts()

Nepal                      20923
USA                         7448
Japan                       6604
UK                          5818
France                      5089
Germany                     3792
Spain                       3598
S Korea                     3006
Italy                       2937
China                       2901
Switzerland                 2656
India                       2469
Austria                     2275
Russia                      1681
Australia                   1641
Poland                      1638
Canada                      1350
New Zealand                  853
Netherlands                  844
Czech Republic               769
Norway                       678
Slovenia                     575
Belgium                      505
Yugoslavia                   493
Ukraine                      459
Iran                         454
Sweden                       425
Czechoslovakia               355
Mexico                       350
Ireland                      296
Argentina 

In [166]:
import re

# Standardize 'citizen' values by sorting country names and removing special characters
def standardize_citizen(val):
    # Remove special characters except for alphabets, numbers, spaces, and '/'
    val = re.sub(r'[^A-Za-z0-9/ ]+', '', val)
    # Split by '/', sort, and join back if multiple countries
    if '/' in val:
        countries = [c.strip() for c in val.split('/')]
        countries = sorted(set(countries))
        return '/'.join(countries)
    return val.strip()

members_expedition['citizen'] = members_expedition['citizen'].apply(standardize_citizen)

In [170]:
# Still there are some inconsistencies in the 'citizen' column which need to be fixed like below
members_expedition['citizen'] = members_expedition['citizen'].replace({'Nepal/India': 'India/Nepal', 'Nepal/India?': 'India/Nepal',
                                                                       'India?': 'India', 'W Germany': 'Germany',
                                                                       'Malaysi': 'Malaysia',
                                                                       'Iran/W Germany': 'Germany/Iran',
                                                                       'Lativa/USA': 'Latvia/USA'})

In [168]:
# now that we have cleaned the data, let's save the cleaned data to a csv file
# members_expedition.to_csv('E:\Maven_Analytics_Challenges\Everest_Challenge\HimalayanExpeditions\members_expedition.csv', index=False)